In [13]:
# pip install -U pymoo

In [14]:
import numpy as np
import pandas as pd
import json

from data_preprocess import *
from run_ga import *
import matplotlib.pyplot as plt
import pickle

In [15]:
tet_path = "./data/tet.json"
fdr_path = "./data/all_class_mutation/Chart-1_fdr.json"

with open(tet_path, 'r') as f:
    tet_data = json.load(f)


with open(fdr_path, 'r') as f:
    fdr_data = json.load(f)

chart_1_tet = tet_data['Chart-1']
chart_1_fdr = dict()

for key, value in fdr_data['Chart_1'].items():
    chart_1_fdr[key] = value['mutation-score']


chart_1_tet = sorted(chart_1_tet.items())
chart_1_fdr = sorted(chart_1_fdr.items())

In [16]:
tests = [i[0] for i in chart_1_tet]
execution_times = np.array([i[1] for i in chart_1_tet])
fault_detections = np.array([i[1] for i in chart_1_fdr])

test_cases = np.column_stack((execution_times, fault_detections))
adequacy_scores = get_adequacy_scores(fault_detections)

Scaling Method: min_max


In [17]:
bitflip = run_nsga(test_cases, False)


Using 1/N Mutation 
Mean execution Time 5.354044444444446
Mean Fault Detection Rate 2.1805153515536633


In [18]:
adeq = run_nsga_with_adequecy(test_cases, adequacy_scores, False)


 Using Adequacy Score
Mean execution Time 5.319666666666667
Mean Fault Detection Rate 2.073650441112962


In [19]:
datadir = 'data/experiment_result'
for pkl_file in os.listdir(datadir):
        if pkl_file.endswith('adeq.pkl'):
            with open(os.path.join(datadir, pkl_file), 'rb') as f: # load pickle file for projects from merged_data
                adeq_res  = pickle.load(f)
        elif pkl_file.endswith('flip.pkl'):
            with open(os.path.join(datadir, pkl_file), 'rb') as f: # load pickle file for projects from merged_data
               bitflip_res = pickle.load(f)
            plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
            plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
            plt.xlabel("TET")
            plt.ylabel("FDR")
            plt.legend()

In [20]:
# bitflip_res = np.abs(bitflip.F)
# adeq_res = np.abs(adeq.F)

# plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
# plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
# plt.xlabel("TET")
# plt.ylabel("FDR")
# plt.legend()

In [21]:
# with open('data/experiment_result/f_cov_m_all/Chart_1_bitflip.pkl', 'rb') as f:
#     bitflip = pickle.load(f)

# with open('data/experiment_result/f_cov_m_cov/Chart_1_adeq.pkl', 'rb') as f:
#     adeq = pickle.load(f)
# # print(data.F)

# bitflip_res = bitflip_res = np.abs(bitflip.F)
# adeq_res = np.abs(adeq.F)

# plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
# plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
# plt.xlabel("TET")
# plt.ylabel("Effectiveness")
# plt.legend()

In [22]:
types = [
    # 'f_cov_m_cov',
    # 'f_cov_m_fdr',
    # 'f_cov_m_flc',
    # 'f_cov_m_latest',
    'f_fdr_m_cov',
    'f_fdr_m_fdr',
    'f_fdr_m_flc',
    # 'f_fdr_m_latest'
]

tet_dict = {}
for type in types:
    print(type)
    for target in os.listdir('./data/merged_data'):
        pid, vid = os.path.splitext(target)[0].split('_')

        with open(f'data/experiment_result/{type}/{pid}_{vid}_bitflip.pkl', 'rb') as f:
            bitflip = pickle.load(f)

        with open(f'data/experiment_result/{type}/{pid}_{vid}_adeq.pkl', 'rb') as f:
            adeq = pickle.load(f)

        with open(f'data/merged_data/{pid}_{vid}.pkl', 'rb') as f:
            data = pickle.load(f)

        with open(f"data/failing_tests/{pid}-{vid}/failing_tests", "r") as f:
            failing_tests = f.readlines()

        min_id_adeq = None
        for i, sol in enumerate(adeq.X):
            # print(data.index[sol].shape)
            selected_tests = data.index[sol]
            if all(test in selected_tests for test in failing_tests):
                tet = adeq.F[i, 0]
                if min_id_adeq == None:
                    min_id_adeq = i
                elif tet < adeq.F[min_id_adeq, 0]:
                    min_id_adeq = i

        min_id_bitflip = None
        for i, sol in enumerate(bitflip.X):
            # print(data.index[sol].shape)
            selected_tests = data.index[sol]
            if all(test in selected_tests for test in failing_tests):
                tet = bitflip.F[i, 0]
                if min_id_bitflip == None:
                    min_id_bitflip = i
                elif tet < bitflip.F[min_id_bitflip, 0]:
                    min_id_bitflip = i

        # print("Adeq tet")
        # print(adeq.F[min_id_adeq, 0])
        # print("Bitflip tet")
        # print(bitflip.F[min_id_bitflip, 0])
        # print("\n")
        tet_dict[f"{pid}_{vid}"] = {"adeq": adeq.F[min_id_adeq, 0], "bigflip": bitflip.F[min_id_bitflip, 0]}
    # with
    print(type)
    print(tet_dict)
    with open(f"./data/experiment_result/{type}/min_tet_with_failing_tests.json", 'w') as f:
        json.dump(tet_dict, f)


f_fdr_m_cov
f_fdr_m_cov
{'JacksonCore_26': {'adeq': 1.9059999999999997, 'bigflip': 1.8890000000000002}, 'Chart_1': {'adeq': 2.316, 'bigflip': 2.0709999999999997}, 'Lang_1': {'adeq': 0.09400000000000001, 'bigflip': 0.11699999999999999}, 'Csv_16': {'adeq': 0.21, 'bigflip': 0.05500000000000001}, 'Time_1': {'adeq': 1.624, 'bigflip': 1.5510000000000002}, 'Cli_40': {'adeq': 0.0, 'bigflip': 0.0}, 'JxPath_22': {'adeq': 1.113, 'bigflip': 1.2150000000000003}}
f_fdr_m_fdr
f_fdr_m_fdr
{'JacksonCore_26': {'adeq': 1.6059999999999999, 'bigflip': 1.8890000000000002}, 'Chart_1': {'adeq': 2.065, 'bigflip': 2.0709999999999997}, 'Lang_1': {'adeq': 0.121, 'bigflip': 0.11699999999999999}, 'Csv_16': {'adeq': 0.429, 'bigflip': 0.05500000000000001}, 'Time_1': {'adeq': 2.9210000000000003, 'bigflip': 1.5510000000000002}, 'Cli_40': {'adeq': 0.0, 'bigflip': 0.0}, 'JxPath_22': {'adeq': 2.0250000000000004, 'bigflip': 1.2150000000000003}}
f_fdr_m_flc
f_fdr_m_flc
{'JacksonCore_26': {'adeq': 2.2540000000000004, 'bigfli

TypeError: Object of type ndarray is not JSON serializable